In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import the data and 

In [ ]:
test = pd.read_csv("../input/demand-forecasting-kernels-only/test.csv")
train = pd.read_csv("../input/demand-forecasting-kernels-only/train.csv")
submission = pd.read_csv("../input/demand-forecasting-kernels-only/sample_submission.csv")

In [ ]:
train.describe()

From the Data we can tell that different time, store and items will lead to a different sales. 

* Date: based on each day from 01/01/2013 - 12/31/2017 
* Store: Integer that represents different stores 
* Item: Integer represents different items 



In [ ]:
#Convert date to datetime format
train['date']=pd.to_datetime(train['date'])
test['date']=pd.to_datetime(test['date'])

#To better understand the daytime, extract the dayofweek, dayofyear, year, month to expand the analyzing variables.


train['weekday']=train['date'].dt.dayofweek
train['dayofyear']=train['date'].dt.dayofyear
train['year']=train['date'].dt.year
train['month']=train['date'].dt.month

#Do the same for the test dataset

test['weekday']=test['date'].dt.dayofweek
test['dayofyear']=test['date'].dt.dayofyear
test['year']=test['date'].dt.year
test['month']=test['date'].dt.month

In [ ]:
import plotly.express as px
px.line(x=train['date'],y=train['sales'],title="Daily Sales")

From the chart, we can tell that the overall sale is incresaing by year. Also the sale is infected by season.

In [ ]:
#Check the current model
train

In [ ]:
#Create trainning and testing datasets for model trainning.
X=train.copy()
X.drop(['sales','date'],axis=1,inplace=True)
y=train['sales']

from sklearn.model_selection import train_test_split

#Splitting data into training and validation test
X_train,X_val,y_train,y_val=train_test_split(X,y, test_size=0.2, random_state=123)

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

xgb=XGBRegressor()


In [ ]:
#define the Smape_scoring
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


In [ ]:
#From this step, we are going to try different model parameters.

In [ ]:
xgb

In [ ]:
xgb.fit(X_train,y_train)
training_predictions=xgb.predict(X_train)
print("SMAPE score:",smape(y_train,training_predictions))

In [ ]:
val=xgb.predict(X_val)
print("SMAPE score:",smape(y_val,val))

Use the test dataset to make default predictions

In [ ]:
X=test.copy()
X.drop(['id','date'],axis=1,inplace=True)

test_predictions=xgb.predict(X)

In [ ]:
submission['sales'] = test_predictions
submission.to_csv('submission.csv', index=False)